In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [4]:
df = pd.read_csv("/content/drive/MyDrive/datasets/aml/Dataset-SA.csv")

df.head()

,product_name,product_price,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,neutral


In [5]:
df = df.drop(["product_name", "product_price", "Rate"],axis=1)
df.head()

,Review,Summary,Sentiment
0,super!,great cooler excellent air flow and for this p...,positive
1,awesome,best budget 2 fit cooler nice cooling,positive
2,fair,the quality is good but the power of air is de...,positive
3,useless product,very bad product its a only a fan,negative
4,fair,ok ok product,neutral


In [6]:
df["input_text"]=  df.Summary+df.Review.fillna("")
df=df[~ df.input_text.isna()]

In [7]:
df.head()

,Review,Summary,Sentiment,input_text
0,super!,great cooler excellent air flow and for this p...,positive,great cooler excellent air flow and for this p...
1,awesome,best budget 2 fit cooler nice cooling,positive,best budget 2 fit cooler nice coolingawesome
2,fair,the quality is good but the power of air is de...,positive,the quality is good but the power of air is de...
3,useless product,very bad product its a only a fan,negative,very bad product its a only a fanuseless product
4,fair,ok ok product,neutral,ok ok productfair


In [8]:
from transformers import DistilBertTokenizerFast

In [9]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [10]:
print(df.Sentiment.value_counts())

positive    166575
negative     28232
neutral      10234
Name: Sentiment, dtype: int64


In [11]:
print(df.shape[0])

205041


In [12]:
df.Sentiment.replace({'positive':2, 'neutral': 1, 'negative': 0}, inplace = True)

In [13]:
from sklearn.model_selection import train_test_split 

train_data, val_data= train_test_split(df, test_size=0.3)
val_data, test_data= train_test_split(val_data, test_size=2/3)

In [14]:
train_data = train_data.reset_index(drop = True)
val_data = val_data.reset_index(drop = True)
test_data = test_data.reset_index(drop = True)
train_data.head()

,Review,Summary,Sentiment,input_text
0,value-for-money,very nice,2,very nicevalue-for-money
1,brilliant,good,2,goodbrilliant
2,terrific,comfortable,2,comfortableterrific
3,awesome,good,2,goodawesome
4,really nice,the delivery is very good and fast the is the ...,2,the delivery is very good and fast the is the ...


In [15]:
train_data_X = train_data['input_text']
val_data_X = val_data['input_text']
test_data_X = test_data['input_text']
train_data_X.head()

0                             very nicevalue-for-money
1                                        goodbrilliant
2                                  comfortableterrific
3                                          goodawesome
4    the delivery is very good and fast the is the ...
Name: input_text, dtype: object

In [16]:
train_data_Y = train_data['Sentiment']
val_data_Y = val_data['Sentiment']
test_data_Y = test_data['Sentiment']

In [17]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(train_data_Y), y = train_data_Y)

class_weights_dict = {}

for x,y in enumerate(class_weights):
  class_weights_dict[x] = y

In [18]:
class_weights_dict

{0: 2.4145890111369064, 1: 6.7289263947491795, 2: 0.41030038992373047}

In [19]:
# Encoding the Training data
train_count=0
train_input_ids=[]
train_attention_masks=[]

val_count=0
val_input_ids=[]
val_attention_masks=[]

test_count=0
test_input_ids=[]
test_attention_masks=[]

while train_count <= len(list(train_data.input_text.values)):
  encoded_data_train = tokenizer.batch_encode_plus(
      list(train_data.input_text.values)[train_count:train_count+64], 
      add_special_tokens=True, 
      return_attention_mask=True, 
      pad_to_max_length=True, 
      max_length=256, 
      return_token_type_ids=False,
      truncation=True
  )

  train_input_ids += encoded_data_train['input_ids']
  train_attention_masks += encoded_data_train['attention_mask']

  train_count += 64

while val_count <= len(list(val_data.input_text.values)):
  # Encoding the Validation data
  encoded_data_val = tokenizer.batch_encode_plus(
      list(val_data.input_text.values)[val_count:val_count+64], 
      add_special_tokens=True, 
      return_attention_mask=True, 
      pad_to_max_length=True, 
      max_length=256, 
      return_token_type_ids=False,
      truncation=True
  )

  val_input_ids += encoded_data_val['input_ids']
  val_attention_masks += encoded_data_val['attention_mask']

  val_count += 64

while test_count <= len(list(test_data.input_text.values)):
  encoded_data_test = tokenizer.batch_encode_plus(
      list(test_data.input_text.values)[test_count:test_count+64], 
      add_special_tokens=True, 
      return_attention_mask=True, 
      pad_to_max_length=True, 
      max_length=256, 
      return_token_type_ids=False,
      truncation=True
  )

  test_input_ids += encoded_data_test['input_ids']
  test_attention_masks += encoded_data_test['attention_mask']

  test_count += 64

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [20]:
train_input_ids = tf.convert_to_tensor(train_input_ids)
val_input_ids = tf.convert_to_tensor(val_input_ids)
test_input_ids = tf.convert_to_tensor(test_input_ids)

train_attention_masks += tf.convert_to_tensor(train_attention_masks)
val_attention_masks += tf.convert_to_tensor(val_attention_masks)
test_attention_masks += tf.convert_to_tensor(test_attention_masks)

In [21]:
from transformers import TFDistilBertModel, DistilBertConfig

distilBert = TFDistilBertModel.from_pretrained('distilbert-base-uncased', config = DistilBertConfig(
    output_hidden_states = True,
    dropout = 0.25,
    attention_dropout = 0.25,
))


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [22]:
for layer in distilBert.layers:
  layer.trainable = False

In [23]:
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import Model

input_ids_layer = Input(shape=(256,), dtype='int32', name="input_ids")
input_mask_layer = Input(shape=(256,), dtype='int32', name="attention_mask")

In [24]:
hs = distilBert([input_ids_layer, input_mask_layer])[0]
cls = hs[:, 0, :]

In [25]:
output_layer = Dense(3,activation = 'softmax')(cls)

In [26]:
model = Model([input_ids_layer, input_mask_layer], output_layer)

In [27]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy

model.compile(tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon = 1e-8), loss='sparse_categorical_crossentropy',metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 BertModel)                     ast_hidden_state=(N               'attention_mask[0][0]']         
                                one, 256, 768),                                                   
                                 hidden_states=((No                                           

In [29]:
model_checkpoint= tf.keras.callbacks.ModelCheckpoint(
    "/content/drive/MyDrive/datasets/aml/dist_bert.h5",
    monitor = "val_loss",
    mode= "auto",
    save_freq="epoch",
    save_best_only = True,
    save_weights_only= True
)

In [ ]:
hist1 = model.fit(
    x = [train_input_ids,train_attention_masks],
    y = train_data_Y.to_numpy(),
    validation_data = ([val_input_ids, val_attention_masks], val_data_Y.to_numpy()),
    batch_size = 64,
    epochs = 8,
    class_weight = class_weights_dict,
    callbacks = [model_checkpoint]
)

Epoch 1/8
2243/2243 [==============================] - 1603s 700ms/step - loss: 0.8440 - accuracy: 0.6749 - val_loss: 0.6990 - val_accuracy: 0.7549
Epoch 2/8
2243/2243 [==============================] - 1546s 689ms/step - loss: 0.7964 - accuracy: 0.7064 - val_loss: 0.6330 - val_accuracy: 0.7875
Epoch 3/8
2243/2243 [==============================] - 1568s 699ms/step - loss: 0.7678 - accuracy: 0.7224 - val_loss: 0.6330 - val_accuracy: 0.7657
Epoch 4/8
2243/2243 [==============================] - 1568s 699ms/step - loss: 0.7504 - accuracy: 0.7334 - val_loss: 0.5868 - val_accuracy: 0.8030
Epoch 5/8
2243/2243 [==============================] - 1567s 698ms/step - loss: 0.7327 - accuracy: 0.7446 - val_loss: 0.5752 - val_accuracy: 0.8064
Epoch 6/8
2243/2243 [==============================] - 1568s 699ms/step - loss: 0.7235 - accuracy: 0.7499 - val_loss: 0.5628 - val_accuracy: 0.8123
Epoch 7/8
2243/2243 [==============================] - 1568s 699ms/step - loss: 0.7121 - accuracy: 0.7539 - val_

In [ ]:
model.save_weights("/content/drive/MyDrive/datasets/aml/dit_bert_hist1.h5")

In [30]:
model.load_weights("/content/drive/MyDrive/datasets/aml/dist_bert_hist1.h5")

In [32]:
for layer in distilBert.layers:
  layer.trainable = True

In [33]:
model.compile(tf.keras.optimizers.Adam(learning_rate=1e-5, epsilon = 1e-8), loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [34]:
hist2 = model.fit(
    x = [train_input_ids,train_attention_masks],
    y = train_data_Y.to_numpy(),
    validation_data = ([val_input_ids, val_attention_masks], val_data_Y.to_numpy()),
    batch_size = 64,
    epochs = 3,
    class_weight = class_weights_dict,
    callbacks = [model_checkpoint]
)

Epoch 1/3
2243/2243 [==============================] - 3584s 2s/step - loss: 0.5043 - accuracy: 0.8518 - val_loss: 0.4095 - val_accuracy: 0.8576
Epoch 2/3
2243/2243 [==============================] - 3527s 2s/step - loss: 0.4863 - accuracy: 0.8545 - val_loss: 0.3162 - val_accuracy: 0.8929
Epoch 3/3
2243/2243 [==============================] - 3524s 2s/step - loss: 0.5032 - accuracy: 0.8522 - val_loss: 0.3453 - val_accuracy: 0.8817


In [35]:
model.save_weights("/content/drive/MyDrive/datasets/aml/dit_bert_hist2.h5")

In [37]:
from sklearn.metrics import classification_report

pred = model.predict({'input_ids': test_input_ids, 'input_mask': test_attention_masks})
y_pred = np.argmax(pred, axis=1)
print(classification_report(test_data_Y, y_pred))

1282/1282 [==============================] - 345s 267ms/step
              precision    recall  f1-score   support

           0       0.74      0.90      0.81      5691
           1       0.33      0.63      0.43      2011
           2       0.99      0.90      0.94     33307

    accuracy                           0.88     41009
   macro avg       0.68      0.81      0.73     41009
weighted avg       0.92      0.88      0.90     41009

